In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
def articles():
    with open('page_revisions_text', 'rb') as text_file:
        pending_article_data = b''
        while True:
            data = text_file.read(1024 ** 2)
            if len(data) == 0:
                break

            articles = data.split(b'\0')
            articles[0] = pending_article_data + articles[0]
            for index, article in enumerate(articles):
                if index + 1 == len(articles):
                    pending_article_data = article
                else:
                    yield article

        print(pending_article_data)
        if len(pending_article_data) != 0:
            yield pending_article_data

In [3]:
subword_text_encoder = tfds.features.text.SubwordTextEncoder.load_from_file('vocab_4096')

Да видим как би изглеждало обучение с кодираните статии...

In [4]:
import os

import numpy as np
import itertools

In [5]:
BATCH_SIZE = 256
BATCHED_ITEM_LENGTH = 256
BUFFER_SIZE = 1024
TYPE=np.int16

def articles_generator():
    for index, article in enumerate(itertools.islice(articles(), 0, 2000)):
        yield np.array(subword_text_encoder.encode(article + b'\0'), dtype=TYPE)

    # Pad the article count to the batch size
    # We do this to ensure that no data is dropped
    index += 1
    while index % BATCH_SIZE != 0:
        yield np.array([0], dtype=TYPE)
        index += 1

def subbatches():
    dataset = tf.data.Dataset.from_generator(articles_generator, output_types=TYPE)
    dataset = dataset.shuffle(BUFFER_SIZE)
    dataset = dataset.padded_batch(BATCH_SIZE, padded_shapes=([None]), drop_remainder=True)
    dataset = dataset.shuffle(2000)

    for batch in dataset.as_numpy_iterator():
        remaining = batch
        while remaining.shape[1] > 1:
            yield remaining[:, :BATCHED_ITEM_LENGTH + 1]
            remaining = remaining[:, BATCHED_ITEM_LENGTH:]

dataset = tf.data.Dataset.from_generator(subbatches, output_types=TYPE, output_shapes=(BATCH_SIZE, None))
dataset = dataset.map(lambda batch: (batch[:, :-1], batch[:, 1:]))

dataset

<MapDataset shapes: ((256, None), (256, None)), types: (tf.int16, tf.int16)>

In [6]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return tf.keras.Sequential([
        tf.keras.layers.Masking(mask_value=0, batch_input_shape=[batch_size, None]),
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size),
    ])

In [10]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def average_batch_length(true_labels, predictions):
    return tf.shape(true_labels)[1]

model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=512, rnn_units=1024, batch_size=BATCH_SIZE)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.compile(optimizer='adam', loss=loss, metrics=[average_batch_length])

In [11]:
checkpoint_dir = './training_checkpoints' # Directory where the checkpoints will be saved
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}") # Name of the checkpoint files

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

In [12]:
class ModelStateResetter(tf.keras.callbacks.Callback):
    def __init__(self):
        self.last_total_length = 0

    def on_batch_end(self, batch, logs={}):
        average_batch_length = logs.get('average_batch_length', 0)
        total_length = int(round(average_batch_length * (batch + 1)))
        current_batch_length = total_length - self.last_total_length
        self.last_total_length = total_length
        
        if current_batch_length < BATCHED_ITEM_LENGTH:
            self.model.reset_states()
        
model_state_resetter_callback = ModelStateResetter()

In [13]:
total_epochs = 10

for epoch in range(total_epochs):
    print('Epoch %d/%d' % (epoch + 1, total_epochs))
    model.fit(dataset, callbacks=[checkpoint_callback, model_state_resetter_callback])

Epoch 1/10
764/764 [==============================] - 972s 1s/step - loss: 0.1330 - average_batch_length: 254.9843
Epoch 2/10
740/740 [==============================] - 939s 1s/step - loss: 0.1336 - average_batch_length: 254.7297
Epoch 3/10
787/787 [==============================] - 995s 1s/step - loss: 0.1236 - average_batch_length: 254.7878
Epoch 4/10
780/780 [==============================] - 987s 1s/step - loss: 0.1237 - average_batch_length: 255.0282
Epoch 5/10
746/746 [==============================] - 947s 1s/step - loss: 0.1281 - average_batch_length: 255.0013
Epoch 6/10
774/774 [==============================] - 981s 1s/step - loss: 0.1214 - average_batch_length: 254.5607
Epoch 7/10
776/776 [==============================] - 1005s 1s/step - loss: 0.1194 - average_batch_length: 255.1160
Epoch 8/10
801/801 [==============================] - 1042s 1s/step - loss: 0.1136 - average_batch_length: 254.7241
Epoch 9/10
765/765 [==============================] - 1014s 1s/step - loss: 0.

In [14]:
with open('page_revisions_text', 'rb') as text_file:
    data = text_file.read()

article = data.split(b'\0')[120]
del data

encoded_article = np.array(subword_text_encoder.encode(article + b'\0'), dtype=TYPE)

print('Raw:', len(article))
print('Encoded:', len(encoded_article))

Raw: 25541
Encoded: 8222


In [15]:
import ctypes

class Huffman:
    huffman = ctypes.CDLL('x64/Release/huffman')
    
    huffman.create_tree.restype = ctypes.c_void_p
    huffman.destroy_tree.restype = None
    huffman.load_weights.restype = None
    huffman.create_code_string.restype = ctypes.c_char_p
    
    def __init__(self, category_count):
        self.tree = ctypes.c_void_p(self.huffman.create_tree(category_count))

    def __del__(self):
        self.huffman.destroy_tree(self.tree)
        
    def load_weights(self, weights):
        self.huffman.load_weights(self.tree, weights.ctypes.data_as(ctypes.POINTER(ctypes.c_float)))
    
    def get_code_length(self, category):
        return self.huffman.get_code_length(self.tree, category)

    def get_code_zero_count(self, category):
        return self.huffman.get_code_zero_count(self.tree, category)

In [16]:
def huffman_archive_size(model, text):
    archived_size = 0
    zeros = 0
    input_eval = np.array([[0]], dtype=TYPE)
    huffman_tree = Huffman(subword_text_encoder.vocab_size)

    text_generated = []

    model.reset_states()

    for index, byte in enumerate(text):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) # remove the batch dimension

        weights = tf.nn.softmax(predictions[0]).numpy()
        huffman_tree.load_weights(weights)
        zeros += huffman_tree.get_code_zero_count(byte.item())
        archived_size += huffman_tree.get_code_length(byte.item())

        input_eval = tf.expand_dims([byte], 0)
  
    return archived_size, zeros

In [17]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size = subword_text_encoder.vocab_size, embedding_dim=512, rnn_units=1024, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [18]:
total_raw = 0
total_compressed = 0

for index, article in enumerate(articles()):
    raw = (len(article) + 1) * 8
    encoded_article = np.array(subword_text_encoder.encode(article + b'\0'), dtype=TYPE)
    compressed, _ = huffman_archive_size(model, encoded_article)
    total_raw += raw
    total_compressed += compressed
    print('Article %d:\tCompression: %f\tAvg Compression: %f' % (index, compressed/raw, total_compressed/total_raw))

Article 0:	Compression: 0.687500	Avg Compression: 0.687500
Article 1:	Compression: 0.419643	Avg Compression: 0.500000
Article 2:	Compression: 0.377551	Avg Compression: 0.444954
Article 3:	Compression: 0.500000	Avg Compression: 0.456835
Article 4:	Compression: 0.421429	Avg Compression: 0.449713
Article 5:	Compression: 0.310897	Avg Compression: 0.424296
Article 6:	Compression: 0.093392	Avg Compression: 0.094592
Article 7:	Compression: 0.324324	Avg Compression: 0.094736
Article 8:	Compression: 0.368750	Avg Compression: 0.094922
Article 9:	Compression: 0.321429	Avg Compression: 0.095084
Article 10:	Compression: 0.384868	Avg Compression: 0.095271
Article 11:	Compression: 0.289773	Avg Compression: 0.095416
Article 12:	Compression: 0.298295	Avg Compression: 0.095567
Article 13:	Compression: 0.315789	Avg Compression: 0.095709
Article 14:	Compression: 0.260638	Avg Compression: 0.095840
Article 15:	Compression: 0.510417	Avg Compression: 0.096008
Article 16:	Compression: 0.446429	Avg Compression:

Article 138:	Compression: 0.205645	Avg Compression: 0.096868
Article 139:	Compression: 0.097996	Avg Compression: 0.096932
Article 140:	Compression: 0.102243	Avg Compression: 0.097282
Article 141:	Compression: 0.117668	Avg Compression: 0.097437
Article 142:	Compression: 0.089504	Avg Compression: 0.097303
Article 143:	Compression: 0.110733	Avg Compression: 0.097428
Article 144:	Compression: 0.307692	Avg Compression: 0.097445
Article 145:	Compression: 0.093106	Avg Compression: 0.097291
Article 146:	Compression: 0.099513	Avg Compression: 0.097327
Article 147:	Compression: 0.104385	Avg Compression: 0.097516
Article 148:	Compression: 0.091199	Avg Compression: 0.097149
Article 149:	Compression: 0.091165	Avg Compression: 0.096771
Article 150:	Compression: 0.475000	Avg Compression: 0.096794
Article 151:	Compression: 0.090699	Avg Compression: 0.096722
Article 152:	Compression: 0.098866	Avg Compression: 0.096918
Article 153:	Compression: 0.104994	Avg Compression: 0.097103
Article 154:	Compression

Article 273:	Compression: 0.119098	Avg Compression: 0.097389
Article 274:	Compression: 0.108525	Avg Compression: 0.097404
Article 275:	Compression: 0.124194	Avg Compression: 0.097423
Article 276:	Compression: 0.089968	Avg Compression: 0.097414
Article 277:	Compression: 0.095108	Avg Compression: 0.097401
Article 278:	Compression: 0.096629	Avg Compression: 0.097397
Article 279:	Compression: 0.108019	Avg Compression: 0.097464
Article 280:	Compression: 0.515625	Avg Compression: 0.097469
Article 281:	Compression: 0.095052	Avg Compression: 0.097419
Article 282:	Compression: 0.101137	Avg Compression: 0.097427
Article 283:	Compression: 0.108087	Avg Compression: 0.097526
Article 284:	Compression: 0.109555	Avg Compression: 0.097683
Article 285:	Compression: 0.383152	Avg Compression: 0.097690
Article 286:	Compression: 0.375000	Avg Compression: 0.097695
Article 287:	Compression: 0.122880	Avg Compression: 0.097733
Article 288:	Compression: 0.115073	Avg Compression: 0.097750
Article 289:	Compression

Article 408:	Compression: 0.114959	Avg Compression: 0.097198
Article 409:	Compression: 0.084732	Avg Compression: 0.097190
Article 410:	Compression: 0.102485	Avg Compression: 0.097202
Article 411:	Compression: 0.564815	Avg Compression: 0.097206
Article 412:	Compression: 0.093450	Avg Compression: 0.097189
Article 413:	Compression: 0.362500	Avg Compression: 0.097191
Article 414:	Compression: 0.090312	Avg Compression: 0.097116
Article 415:	Compression: 0.159161	Avg Compression: 0.097132
Article 416:	Compression: 0.097190	Avg Compression: 0.097132
Article 417:	Compression: 0.099162	Avg Compression: 0.097135
Article 418:	Compression: 0.100356	Avg Compression: 0.097143
Article 419:	Compression: 0.083725	Avg Compression: 0.096833
Article 420:	Compression: 0.096980	Avg Compression: 0.096834
Article 421:	Compression: 0.109188	Avg Compression: 0.096878
Article 422:	Compression: 0.109441	Avg Compression: 0.096885
Article 423:	Compression: 0.132026	Avg Compression: 0.096893
Article 424:	Compression

Article 543:	Compression: 0.104821	Avg Compression: 0.096946
Article 544:	Compression: 0.077709	Avg Compression: 0.096600
Article 545:	Compression: 0.393382	Avg Compression: 0.096602
Article 546:	Compression: 0.088481	Avg Compression: 0.096578
Article 547:	Compression: 0.097938	Avg Compression: 0.096580
Article 548:	Compression: 0.096743	Avg Compression: 0.096580
Article 549:	Compression: 0.089395	Avg Compression: 0.096551
Article 550:	Compression: 0.117379	Avg Compression: 0.096569
Article 551:	Compression: 0.105829	Avg Compression: 0.096592
Article 552:	Compression: 0.094348	Avg Compression: 0.096587
Article 553:	Compression: 0.576389	Avg Compression: 0.096589
Article 554:	Compression: 0.109168	Avg Compression: 0.096614
Article 555:	Compression: 0.102836	Avg Compression: 0.096625
Article 556:	Compression: 0.523438	Avg Compression: 0.096628
Article 557:	Compression: 0.098888	Avg Compression: 0.096628
Article 558:	Compression: 0.105462	Avg Compression: 0.096642
Article 559:	Compression

...

Article 6551:	Compression: 0.262843	Avg Compression: 0.263809
Article 6552:	Compression: 0.269038	Avg Compression: 0.263811
Article 6553:	Compression: 0.250129	Avg Compression: 0.263806
Article 6554:	Compression: 0.252924	Avg Compression: 0.263803
Article 6555:	Compression: 0.253500	Avg Compression: 0.263801
Article 6556:	Compression: 0.260707	Avg Compression: 0.263800
Article 6557:	Compression: 0.260180	Avg Compression: 0.263799
Article 6558:	Compression: 0.280164	Avg Compression: 0.263800
Article 6559:	Compression: 0.347212	Avg Compression: 0.263807
Article 6560:	Compression: 0.316427	Avg Compression: 0.263821
Article 6561:	Compression: 0.636364	Avg Compression: 0.263821
Article 6562:	Compression: 0.560484	Avg Compression: 0.263821
Article 6563:	Compression: 0.328747	Avg Compression: 0.263838
Article 6564:	Compression: 0.312197	Avg Compression: 0.263843
Article 6565:	Compression: 0.419643	Avg Compression: 0.263843
Article 6566:	Compression: 0.254125	Avg Compression: 0.263840
Article 

Article 6684:	Compression: 0.307478	Avg Compression: 0.265765
Article 6685:	Compression: 0.335869	Avg Compression: 0.265775
Article 6686:	Compression: 0.385449	Avg Compression: 0.265831
Article 6687:	Compression: 0.512931	Avg Compression: 0.265831
Article 6688:	Compression: 0.315513	Avg Compression: 0.265836
Article 6689:	Compression: 0.458752	Avg Compression: 0.265876
Article 6690:	Compression: 0.676724	Avg Compression: 0.265877
Article 6691:	Compression: 0.317348	Avg Compression: 0.265904
Article 6692:	Compression: 0.328832	Avg Compression: 0.265946
Article 6693:	Compression: 0.391933	Avg Compression: 0.265949
Article 6694:	Compression: 0.464286	Avg Compression: 0.265949
Article 6695:	Compression: 0.296755	Avg Compression: 0.265960
Article 6696:	Compression: 0.232395	Avg Compression: 0.265956
Article 6697:	Compression: 0.398026	Avg Compression: 0.265957
Article 6698:	Compression: 0.297468	Avg Compression: 0.265967
Article 6699:	Compression: 0.285081	Avg Compression: 0.265972
Article 

Article 6817:	Compression: 0.370079	Avg Compression: 0.267637
Article 6818:	Compression: 0.254030	Avg Compression: 0.267634
Article 6819:	Compression: 0.414326	Avg Compression: 0.267658
Article 6820:	Compression: 0.406370	Avg Compression: 0.267769
Article 6821:	Compression: 0.296107	Avg Compression: 0.267773
Article 6822:	Compression: 0.346384	Avg Compression: 0.267786
Article 6823:	Compression: 0.562500	Avg Compression: 0.267786
Article 6824:	Compression: 0.320421	Avg Compression: 0.267798
Article 6825:	Compression: 0.718750	Avg Compression: 0.267799
Article 6826:	Compression: 0.380597	Avg Compression: 0.267810
Article 6827:	Compression: 0.263370	Avg Compression: 0.267809
Article 6828:	Compression: 0.290799	Avg Compression: 0.267814
Article 6829:	Compression: 0.362932	Avg Compression: 0.267822
Article 6830:	Compression: 0.296952	Avg Compression: 0.267823
Article 6831:	Compression: 0.328490	Avg Compression: 0.267832
Article 6832:	Compression: 0.537500	Avg Compression: 0.267832
Article 

Article 6950:	Compression: 0.332718	Avg Compression: 0.269018
Article 6951:	Compression: 0.355787	Avg Compression: 0.269040
Article 6952:	Compression: 0.341936	Avg Compression: 0.269042
Article 6953:	Compression: 0.249570	Avg Compression: 0.269038
Article 6954:	Compression: 0.783333	Avg Compression: 0.269038
Article 6955:	Compression: 0.367891	Avg Compression: 0.269039
Article 6956:	Compression: 0.593750	Avg Compression: 0.269039
Article 6957:	Compression: 0.572115	Avg Compression: 0.269039
Article 6958:	Compression: 0.257017	Avg Compression: 0.269037
Article 6959:	Compression: 0.814815	Avg Compression: 0.269037
Article 6960:	Compression: 0.724138	Avg Compression: 0.269037
Article 6961:	Compression: 0.278275	Avg Compression: 0.269037
Article 6962:	Compression: 0.321009	Avg Compression: 0.269039
Article 6963:	Compression: 0.306854	Avg Compression: 0.269045
Article 6964:	Compression: 0.442005	Avg Compression: 0.269110
Article 6965:	Compression: 0.420833	Avg Compression: 0.269110
Article 

KeyboardInterrupt: 